In [18]:
import pandas as pd
import nltk 
import numpy as np
import re
from nltk.stem import wordnet # to perform lemmitization
from sklearn.feature_extraction.text import CountVectorizer # to perform bow
from sklearn.feature_extraction.text import TfidfVectorizer # to perform tfidf
from nltk import pos_tag # for parts of speech
from sklearn.metrics import pairwise_distances # to perfrom cosine similarity
from nltk import word_tokenize # to create tokens
from nltk.corpus import stopwords # for stop words

def loadxlxs():
    global df
    df=pd.read_excel('dialog_talk_agent.xlsx')
    df.ffill(axis = 0,inplace=True)

    
# function that performs text normalization steps

def text_normalization(text):


    text=str(text).lower() # text to lower case
    spl_char_text=re.sub(r'[^ a-z]','',text) # removing special characters
    tokens=nltk.word_tokenize(spl_char_text) # word tokenizing
    lema=wordnet.WordNetLemmatizer() # intializing lemmatization
    tags_list=pos_tag(tokens,tagset=None) # parts of speech
    lema_words=[]   # empty list 
    for token,pos_token in tags_list:
        if pos_token.startswith('V'):  # Verb
            pos_val='v'
        elif pos_token.startswith('J'): # Adjective
            pos_val='a'
        elif pos_token.startswith('R'): # Adverb
            pos_val='r'
        else:
            pos_val='n' # Noun
        lema_token=lema.lemmatize(token,pos_val) # performing lemmatization
        lema_words.append(lema_token) # appending the lemmatized token into a list
    
    return " ".join(lema_words) # returns the lemmatized tokens as a sentence 


def chat_tfidf(text):
    tfidf=TfidfVectorizer() # intializing tf-id 
    x_tfidf=tfidf.fit_transform(df['lemmatized_text']).toarray()
    df_tfidf=pd.DataFrame(x_tfidf,columns=tfidf.get_feature_names()) 
    lemma=text_normalization(text) # calling the function to perform text normalization
    tf=tfidf.transform([lemma]).toarray() # applying tf-idf
    cos=1-pairwise_distances(df_tfidf,tf,metric='cosine') # applying cosine similarity
    index_value=cos.argmax() # getting index value 
    return df['Text Response'].loc[index_value]

#初始化
loadxlxs()
df['lemmatized_text']=df['Context'].apply(text_normalization)

#问答
text = input()
print(chat_tfidf(text))

cancer
This website could help you: https://www.cdc.gov/cancer/breast/basic_info/symptoms.htm


In [20]:
text = input()
print(chat_tfidf(text))

doctor
This website could help you: https://www.nationalbreastcancer.org/breast-cancer-doctors
